БИБЛИОТЕКИ

In [ ]:
import numpy as np
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import recall_score
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split

ЧТЕНИЕ МАССИВОВ

In [ ]:
# монтируем свой диск...
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


ССЫЛКИ НА ФАЙЛЫ ДЛЯ ШАГА 1 - ОТСЕЧЕНИЕ КЛАССА '0000'

In [ ]:
# входные данные
x_train_file = '/content/drive/My Drive/Bases/DTP/x_train_0.dat'
y_train_file = '/content/drive/My Drive/Bases/DTP/y_train_0.dat'

# выходный файл с весами лучшей модели
model_save_file = '/content/drive/My Drive/Bases/DTP/model_0.h5'

ЧТЕНИЕ ДАННЫХ

In [ ]:
# чтение
x_train = np.fromfile(x_train_file)
x_train = x_train.reshape((35771, 7))
y_train = np.fromfile(y_train_file)

# 5% на проверочную выборку
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.05)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(33982, 7)
(33982,)
(1789, 7)
(1789,)


НАСТРОЙКА ВЕСОВ БИНАРНОЙ КЛАССИФИКАЦИИ

In [ ]:
# веса модели
weight_for_0 = 1.0 / (y_train == 0.0).sum()
weight_for_1 = 1.0 / (y_train == 1.0).sum()

class_weight = {0: weight_for_0, 1: weight_for_1}

print(class_weight)

{0: 3.396393030601501e-05, 1: 0.0002203128442388191}


МОДЕЛЬ НЕЙРОННОЙ СЕТИ

In [ ]:
def CreateModel(activation='linear', dropout=0.1):

  input = Input((7,)) 

  x = BatchNormalization()(input)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)
  
  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  output = Dense(1, activation='sigmoid')(x) 
  
  return Model(input, output) 

CALLBACK, сохраняющий лучший результат по метрике Recall

In [ ]:
class MyCallback(Callback):
  
  # инициализация
  def __init__(self, model, x_val, y_val):
    # модель
    self.model = model 
    # проверочная выборка
    self.x_val = x_val 
    self.y_val = y_val
    # начальный счет по метрике
    self.score = 0
       
  # подсчет метрики Recall для val  
  def calc_recall(self):
    y_pred = self.model.predict(self.x_val)
    y_p = np.around(y_pred).astype(int)
    y_t = np.around(self.y_val).astype(int)
    return recall_score(y_t, y_p, average='macro')

  # окончение каждой эпохи....
  def on_epoch_end(self, epoch, logs=None):
    score_0 = self.calc_recall()
    print('\n\nТекущий Recall =', score_0, '\n')
    if score_0 > self.score:
      self.score = score_0
      print('\n\n!!!!!!!!Лучший Recall =', score_0, '\n')
      self.model.save_weights(model_save_file)   

  # в конце обучения
  def on_train_end(self, logs=None):
    print('\n\n!ЛУЧШИЙ Recall ОБУЧЕНИЯ =', self.score, '\n')


In [ ]:
# параметры обучения

batch_size = 1000

dropout = 0.1

activate_function = 'relu'

epochs_count = 300

learning_rate = 1e-4

ОБУЧЕНИЕ МОДЕЛИ БИНАРНОЙ КЛАССИФИКАЦИИ ШАГ 1

In [ ]:
del model

In [ ]:
# создание модели
model = CreateModel(activation=activate_function, dropout=dropout)  
cb = MyCallback(model, x_val, y_val)
#cb = MyCallback(model, x_train, y_train)

In [ ]:
# компиляция
model.compile(optimizer=Adam(learning_rate=learning_rate), 
              loss=['binary_crossentropy'], 
              metrics=['accuracy'])

# обучение
history = model.fit(x_train, y_train, 
                    batch_size = batch_size,
                    epochs= epochs_count, 
                    validation_data=(x_val, y_val), 
                    verbose=1, 
                    class_weight = class_weight,
                    callbacks=[cb])

Epoch 1/300
34/34 [==============================] - ETA: 0s - loss: 4.5232e-05 - accuracy: 0.5129

Текущий Recall = 0.4866864624105818 



!!!!!!!!Лучший Recall = 0.4866864624105818 

34/34 [==============================] - 4s 66ms/step - loss: 4.5232e-05 - accuracy: 0.5129 - val_loss: 0.6982 - val_accuracy: 0.4723
Epoch 2/300
33/34 [============================>.] - ETA: 0s - loss: 4.3689e-05 - accuracy: 0.5176

Текущий Recall = 0.4616628424888649 

34/34 [==============================] - 1s 28ms/step - loss: 4.3689e-05 - accuracy: 0.5181 - val_loss: 0.6999 - val_accuracy: 0.4136
Epoch 3/300
33/34 [============================>.] - ETA: 0s - loss: 4.3190e-05 - accuracy: 0.5235

Текущий Recall = 0.4959238763665812 



!!!!!!!!Лучший Recall = 0.4959238763665812 

34/34 [==============================] - 1s 28ms/step - loss: 4.3182e-05 - accuracy: 0.5230 - val_loss: 0.7160 - val_accuracy: 0.3074
Epoch 4/300
32/34 [===========================>..] - ETA: 0s - loss: 4.2705e-05 - accuracy

ПРОВЕРИМ RECALL

In [ ]:
# чтение весов
model.load_weights('/content/drive/My Drive/Bases/DTP/model_0.h5')

In [ ]:
y_pred = model.predict(x_val)
y_p = np.around(y_pred).astype(int)
y_t = np.around(y_val).astype(int)

print(recall_score(y_t, y_p, average='macro'))

0.6014306924011338
